# Ruwaida Rashid

## **To what extent can the distance from last point of transaction made with a specific credit card or the median total spent on a sale be said to contribute to the possibility of credit card fraud?**

I have an interest in data privacy and breaches and with the limited statistical and python knowledge I believe this credit card fraud analysis allows me to gain the basic technical knowledge needed to further understand the techniques involved in further complex data analysis. 
 Following the factors associated with a credit card transaction, I would like to understand if a trend exists between the distance from last location of transaction with the same credit card and a fraudulent transaction. Given the complexity of real life scenarios, I am also interested to know if higher median transaction suggests a greater chance of fraudulent transaction going through.